In [326]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [327]:
df_clean = pd.read_csv('data/df_clean.csv')

# selection des features et de la cible
X = df_clean.drop('charges', axis=1)

y = df_clean.charges



In [328]:
# import du csv
df = pd.read_csv('data/df_clean.csv')

# selection des features et de la cible
X = df.drop('charges', axis=1)

y = df.charges

display(X.dtypes)

# Création du train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=0)

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
dtype: object

In [329]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.linear_model import LinearRegression, Lasso,ElasticNet,Ridge
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from feature_engine.discretisation import ArbitraryDiscretiser

# Séparation des colonnes numériques et catégorielles



cat_col = ['region']

num_col = X.select_dtypes(include = 'number').columns
num_col = num_col[num_col != 'bmi']

ordinal_col= ['sex', 'smoker']

#Transformation de BMI avec FunctionTransformer


# pipeline pour les valeurs numériques
# cela permet de créer une succession d'étapes (ici il n'y en a qu'une)

bmi_dict = {'bmi': [0, 18.5, 24.9, 28.5, 35, 40, np.Inf]}

bmi_pipe = make_pipeline(ArbitraryDiscretiser(binning_dict= bmi_dict, return_object= False, return_boundaries= True), OneHotEncoder())

my_num_pipe = make_pipeline(StandardScaler())

my_cat_pipe = make_pipeline(OneHotEncoder())

my_ordi_pipe = make_pipeline(OrdinalEncoder())

                            
# ColumnTransformer permet de faire des opérations sur des sélections de colonnes

preprocessor = make_column_transformer((bmi_pipe,['bmi']),
                                       (my_ordi_pipe,ordinal_col),  
                                       (my_cat_pipe,cat_col),
                                       
                                       remainder='passthrough'
                                       )


final_preprocessor = make_pipeline(preprocessor,PolynomialFeatures(degree=2),StandardScaler())


# affichage

tab = final_preprocessor.fit(X_train)

cols = tab.get_feature_names_out()

df_transform = pd.DataFrame(final_preprocessor.fit_transform(X_train), columns=cols)
df_transform.head()

,1,"pipeline-1__bmi_(-0.001, 18.5]","pipeline-1__bmi_(18.5, 24.9]","pipeline-1__bmi_(24.9, 28.5]","pipeline-1__bmi_(28.5, 35.0]","pipeline-1__bmi_(35.0, 40.0]","pipeline-1__bmi_(40.0, inf]",pipeline-2__sex,pipeline-2__smoker,pipeline-3__region_northeast,...,pipeline-3__region_southeast^2,pipeline-3__region_southeast pipeline-3__region_southwest,pipeline-3__region_southeast remainder__age,pipeline-3__region_southeast remainder__children,pipeline-3__region_southwest^2,pipeline-3__region_southwest remainder__age,pipeline-3__region_southwest remainder__children,remainder__age^2,remainder__age remainder__children,remainder__children^2
0,0.0,-0.119523,2.252746,-0.505221,-0.776052,-0.456630,-0.273386,1.003527,-0.506594,-0.56244,...,1.635963,0.0,2.274029,2.274155,-0.573284,-0.531603,-0.356947,1.035727,1.257815,0.319401
1,0.0,-0.119523,-0.443903,-0.505221,1.288573,-0.456630,-0.273386,-0.996485,-0.506594,-0.56244,...,-0.611261,0.0,-0.563488,-0.368751,-0.573284,-0.531603,-0.356947,1.132092,1.296837,0.319401
2,0.0,-0.119523,-0.443903,1.979330,-0.776052,-0.456630,-0.273386,1.003527,1.973969,-0.56244,...,-0.611261,0.0,-0.563488,-0.368751,1.744335,1.159691,3.351820,-0.692653,0.965148,1.479376
3,0.0,-0.119523,-0.443903,-0.505221,-0.776052,2.189959,-0.273386,1.003527,1.973969,-0.56244,...,1.635963,0.0,1.380736,0.952702,-0.573284,-0.531603,-0.356947,-0.331948,-0.127475,-0.376583
4,0.0,-0.119523,-0.443903,-0.505221,-0.776052,-0.456630,3.657834,-0.996485,-0.506594,-0.56244,...,-0.611261,0.0,-0.563488,-0.368751,1.744335,0.886902,0.879309,-0.944617,-0.342097,-0.376583


In [330]:
final_preprocessor

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('arbitrarydiscretiser',
                                                                   ArbitraryDiscretiser(binning_dict={'bmi': [0,
                                                                                                              18.5,
                                                                                                              24.9,
                                                                                                              28.5,
                                                                                                              35,
                                                                                                              40,
                                                                                                              inf]},
                                                                                        return_boundaries=True)),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['bmi']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('ordinalencoder',
                                                                   OrdinalEncoder())]),
                                                  ['sex', 'smoker']),
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler())])

In [331]:
from sklearn.model_selection import GridSearchCV

model= make_pipeline(final_preprocessor, ElasticNet(max_iter=1000000))

# Définir les valeurs à tester pour alpha et l1_ratio

param_grid = {'elasticnet__alpha': np.arange(50,100,1),    #np.arange(0.01, 0.1,0.001)
              'elasticnet__l1_ratio':[1]}      #np.arange(0.1,1,0.01)


# creation de l'objet GridSearch

grid_search = GridSearchCV(model,param_grid, cv=5)

# recherche des hyperparamètres et Afficher les meilleures valeurs trouvées

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)

print("Score : ", best_model.score(X_test, y_test))

# calcul RSME

rmse= np.sqrt(mean_squared_error(y_test,best_model.predict(X_test)))
print("RSME: ",rmse)


Meilleurs paramètres :  {'elasticnet__alpha': 99, 'elasticnet__l1_ratio': 1}
Score :  0.8321263359839259
RSME:  5177.546779160623


## Regression Linéaire 

In [332]:

y=df_clean[['charges']]
X = df_clean[['bmi','age','smoker','sex','children']]

# regression lineaire, calcule R2, scatterplot

model = make_pipeline(preprocessor,LinearRegression())
model.fit(X_train,y_train)
model.score(X_train,y_train)
print("R2 (coefficient de détermination): ",model.score(X_test,y_test))


R2 (coefficient de détermination):  0.7342387209828931


## Regression Lasso

In [333]:
model= make_pipeline(final_preprocessor,Lasso(max_iter=1000000))

param_grid={'lasso__alpha':np.arange(0.1,100,10)}

grid_search = GridSearchCV(model,param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)


Meilleurs paramètres :  {'lasso__alpha': 90.1}


0.8323524141942823

## Regression Ridge

In [334]:
model= make_pipeline(preprocessor,Ridge(max_iter=1000000))
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.734120800930419